# INIT

START

In [ ]:
from pydoc import doc
import sys
from pathlib import Path
from multiversx_sdk import ApiNetworkProvider, ProxyNetworkProvider
from multiversx_sdk.abi import Abi
import os
import importlib
from argparse import Namespace
from time import sleep
import pprint

os.environ["MX_DEX_ENV"] = "chainsim"
os.environ["LOG_LEVEL"] = "INFO"

sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
importlib.reload(config)
from context import Context
from utils.utils_chain import WrapperAddress, Account
from utils.utils_generic import get_logger
from tools.chain_simulator_connector import ChainSimulator, start_handler
from contracts.metastaking_contract import MetaStakingContract
from contracts.farm_contract import FarmContract
from contracts.staking_contract import StakingContract
from utils.utils_scenarios import PhaseDictsCollector

docker_path = config.HOME / "Projects/testing/full-stack-docker-compose/chain-simulator"
chain_sim = ChainSimulator(docker_path)

logger = get_logger("farm-timestamps")

if config.CURRENT_ENV.value == "chainsim":
    global found_accounts
    found_accounts = []
    if not chain_sim.is_running():
        state_path = config.DEFAULT_WORKSPACE / "states"
        args = Namespace(docker_path=str(docker_path), state_path=str(state_path))
        chain_sim, found_accounts = start_handler(args)
        print(f'Loaded {len(found_accounts)} accounts')

context = Context()

FARM_BYTECODE_PATH = config.HOME / "Projects/dex/mx-exchange-sc/output-docker/farm-with-locked-rewards/farm-with-locked-rewards.wasm"
FARM_EXPECTED_CODEHASH = "2dc81c4dc7981ee14a8e86fca9f466172d5df6c62e849a270fe7c3dc8572988a"

STAKING_BYTECODE_PATH = config.HOME / "Projects/dex/mx-exchange-sc/output-docker/farm-staking/farm-staking.wasm"
STAKING_EXPECTED_CODEHASH = "eeb701f2b8a7e8ffddef36939f9e7149e14c72b766d75855cdb23caab0097c5e"

In [ ]:
metastaking_contract: MetaStakingContract = context.deploy_structure.get_deployed_contract_by_index(config.METASTAKINGS_BOOSTED, 0)
farm_contract: FarmContract = context.deploy_structure.get_deployed_contract_by_address(config.FARMS_V2, metastaking_contract.farm_address)
staking_contract: StakingContract = context.deploy_structure.get_deployed_contract_by_address(config.STAKINGS_V2, metastaking_contract.stake_address)

print(f"Using {metastaking_contract.address} : {metastaking_contract.metastake_token}")
pprint.pprint(metastaking_contract.get_config_dict())
pprint.pprint(staking_contract.get_config_dict())
pprint.pprint(farm_contract.get_config_dict())

In [ ]:
from utils.utils_scenarios import collect_farm_contract_users, FetchedUser
from typing import List

def collect_users_for_contract(contract_address: str, farming_token: str, farm_token: str, proxy: ProxyNetworkProvider) -> List[FetchedUser]:
    mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
    SEARCH_BATCH_SIZE = 100
    fetched_users = collect_farm_contract_users(SEARCH_BATCH_SIZE, contract_address, farming_token, farm_token,
                                                mainnet_api, proxy)

    users: List[FetchedUser] = fetched_users.get_users_with_farm_tokens()
    fetch_attempts = 0
    while not users and fetch_attempts < 5:
        fetched_users = collect_farm_contract_users(SEARCH_BATCH_SIZE, contract_address, farming_token, farm_token,
                                                    mainnet_api, proxy, fetch_attempts * SEARCH_BATCH_SIZE)
        users: List[FetchedUser] = fetched_users.get_users_with_farm_tokens()
        fetch_attempts += 1
    if not users:
        logger.warning(f"No users found with both tokens for {contract_address}")
        return fetched_users.get_users_with_farm_tokens()
    
    return users

# CHAIN SIM RESTART

In [ ]:
state_path = config.DEFAULT_WORKSPACE / "states"
args = Namespace(docker_path=str(docker_path), state_path=str(state_path))
chain_sim, found_accounts = start_handler(args)
print(f'Loaded {len(found_accounts)} accounts')

# CHAIN CONFIG STATE RETRIEVE - ONE TIME ONLY

In [ ]:
from tools.chain_simulator_connector import retrieve_handler

retrieve_gateway = ProxyNetworkProvider("https://proxy-shadowfork-four.elrond.ro")

collected_users = collect_users_for_contract(metastaking_contract.address, metastaking_contract.farm_token, metastaking_contract.metastake_token, retrieve_gateway)
if not collected_users:
    raise Exception("No users found with both tokens")

for user in collected_users:
    args = Namespace(gateway=retrieve_gateway.url, account=user.address.bech32())
    retrieve_handler(args)

print("Restart chain simulator to load the new state!")

# Functions

CHAIN CONFIG SETUP

In [ ]:
import json
from typing import Any
from utils.utils_chain import WrapperAddress

def chain_sim_init():
    state_path = config.DEFAULT_WORKSPACE / "states"
    args = Namespace(docker_path=str(docker_path), state_path=str(state_path))
    chain_sim, found_accounts = start_handler(args)
    print(f'Loaded {len(found_accounts)} accounts')
    return chain_sim, found_accounts

def advance_blocks(number_of_blocks: int):
    chain_sim.advance_blocks(number_of_blocks)

def advance_epoch(number_of_epochs: int):
    chain_sim.advance_epochs(number_of_epochs)


def users_init() -> list[Account]:
    print(context.deployer_account.address.bech32())
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    users = []
    for user in found_accounts:
        if user == context.deployer_account.address.bech32():   # skip deployer account
            continue
        user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
        user_account.address = WrapperAddress(user)
        if user_account.address.get_shard() != 1:   # select only shard 1 accounts due to limitation in system account token attributes retrieval
            continue
        user_account.sync_nonce(context.network_provider.proxy)
        users.append(user_account)

    return users

CHAIN SIMULATOR STACK

In [ ]:
import subprocess
from time import sleep

CS_DOCKER_PATH = Path.home() / "projects/testing/full-stack-docker-compose/chain-simulator"

def start_chain_sim_stack():
    # stop first in case one is already running
    p = subprocess.Popen(["docker", "compose", "down"], cwd = CS_DOCKER_PATH)
    p.wait()
    
    p = subprocess.Popen(["docker", "compose", "up", "-d"], cwd = CS_DOCKER_PATH)
    sleep(60)
    return p

def stop_chain_sim_stack(p):
    p.terminate()
    p = subprocess.Popen(["docker", "compose", "down"], cwd = CS_DOCKER_PATH)
    p.wait()
    _ = subprocess.run(["docker", "system", "prune", "-f"], cwd = CS_DOCKER_PATH)

Farm upgrade

In [ ]:
def farm_upgrade(assert_code_hash: bool = True):
    tx_hash = farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                            FARM_BYTECODE_PATH, 
                                            [], True)

    # advance_blocks(1)
    tx_hash = farm_contract.resume(context.deployer_account, context.network_provider.proxy)
    # advance_blocks(1)

    if assert_code_hash:
        code_hash = context.network_provider.proxy.get_account(WrapperAddress(farm_contract.address)).contract_code_hash.hex()
        assert code_hash == FARM_EXPECTED_CODEHASH

Metastaking upgrade

In [ ]:
def metastaking_upgrade():
    raise NotImplementedError

Staking upgrade

In [ ]:
def staking_upgrade(assert_code_hash: bool = True):
    tx_hash = staking_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                                STAKING_BYTECODE_PATH,
                                                [], True)

    # advance_blocks(1)
    tx_hash = staking_contract.resume(context.deployer_account, context.network_provider.proxy)
    # advance_blocks(1)

    if assert_code_hash:
        code_hash = context.network_provider.proxy.get_account(WrapperAddress(staking_contract.address)).contract_code_hash.hex()
        assert code_hash == STAKING_EXPECTED_CODEHASH

Deploy permissions hub

In [ ]:
from contracts.permissions_hub_contract import PermissionsHubContract

def deploy_permissions_hub():
    raise NotImplementedError

Dummy proxy

In [ ]:
from contracts.dummy_proxy_contract import DummyProxyContract

def deploy_dummy_proxy_contract():
    dummy_proxy_contract = DummyProxyContract("")
    _, address = dummy_proxy_contract.contract_deploy(context.deployer_account, context.network_provider.proxy,
                                         "https://github.com/ovidiuolteanu/mx-sc-dummy-proxy/releases/download/v2.1/dummy-proxy.wasm",
                                         [])
    dummy_proxy_contract.address = address
    return dummy_proxy_contract

In [ ]:
from utils.utils_chain import get_all_token_nonces_details_for_account

def get_position_for_account(user_address: str):
    metastake_tk_balance, metastake_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, user_address, context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > metastake_tk_balance:
            metastake_tk_balance = int(token['balance'])
            metastake_tk_nonce = token['nonce']
            break

    if not metastake_tk_nonce:
        raise Exception("Not enough metastake token balance")
    
    return metastake_tk_nonce, metastake_tk_balance

def get_farming_position_for_account(user_address: str):
    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_address, context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")
    
    return farm_tk_nonce, farm_tk_balance

In [ ]:
from utils.utils_tx import ESDTToken, multi_esdt_transfer

def send_tokens_to_dummy(user: Account, dummy_contract: DummyProxyContract):
    farm_tk_nonce, farm_tk_balance = get_position_for_account(user)
    print(f"Sending {farm_tk_balance} {metastaking_contract.metastake_token}-{farm_tk_nonce} to dummy contract")
    multi_esdt_transfer(context.network_provider.proxy, 20000000, user, dummy_contract.address, [ESDTToken(metastaking_contract.metastake_token, farm_tk_nonce, farm_tk_balance)])

Claim

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def claim_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough metastake token balance")
    
    tokens = [ESDTToken(metastaking_contract.metastake_token, farm_tk_nonce, farm_tk_balance)]

    tx_hash = metastaking_contract.claim_rewards_metastaking(context.network_provider.proxy, user_account, [tokens])
    return tx_hash

Claim on behalf

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def claim_on_behalf_from_user(claim_account: Account):
    claim_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, claim_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")
    
    tokens = [ESDTToken(metastaking_contract.metastake_token, farm_tk_nonce, farm_tk_balance)]

    tx_hash = metastaking_contract.claim_rewards_on_behalf_metastaking(context.network_provider.proxy, claim_account, [tokens])
    return tx_hash

Enter farm consolidated

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def enter_farm_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance, farming_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.farm_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            farming_tk_nonce = token['nonce']
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")
    
    tokens = [ESDTToken(metastaking_contract.farm_token, farming_tk_nonce, farming_tk_balance)]

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            
            tokens.append(ESDTToken(metastaking_contract.metastake_token, farm_tk_nonce, farm_tk_balance))

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    tx_hash = metastaking_contract.enter_metastake(context.network_provider.proxy, user_account, [tokens])
    return tx_hash

Enter farm no consolidation

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def enter_farm_no_consolidation_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance, farming_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.farm_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            farming_tk_nonce = token['nonce']
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    tokens = [ESDTToken(metastaking_contract.farm_token, farming_tk_nonce, farming_tk_balance)]
    tx_hash = metastaking_contract.enter_metastake(context.network_provider.proxy, user_account, [tokens])
    return tx_hash

Enter farm on behalf

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def enter_farm_on_behalf_for_user(caller_account: Account, user_account: Account):
    caller_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance, farming_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.farm_token, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            farming_tk_nonce = token['nonce']
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")
    
    tokens = [ESDTToken(metastaking_contract.farm_token, farming_tk_nonce, farming_tk_balance)]

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            
            tokens.append(ESDTToken(metastaking_contract.metastake_token, farm_tk_nonce, farm_tk_balance))

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    tx_hash = metastaking_contract.enter_metastake_on_behalf(context.network_provider.proxy, caller_account, [tokens, user_account.address.bech32()])
    return tx_hash

Enter farm on behalf no consolidation

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def enter_farm_on_behalf_no_consolidation_for_user(caller_account: Account, user_account: Account):
    caller_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance, farming_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.farm_token, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            farming_tk_nonce = token['nonce']
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    tokens = [ESDTToken(metastaking_contract.farm_token, farming_tk_nonce, farming_tk_balance)]

    tx_hash = metastaking_contract.enter_metastake_on_behalf(context.network_provider.proxy, caller_account, [tokens, user_account.address.bech32()])
    return tx_hash

Exit farm

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def exit_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough metastake token balance")
    
    tokens = [ESDTToken(metastaking_contract.metastake_token, farm_tk_nonce, farm_tk_balance)]

    tx_hash = metastaking_contract.exit_metastake(context.network_provider.proxy, user_account, [tokens, 1, 1])
    return tx_hash

Utilities

In [ ]:
def get_time_data():
    raw_data = context.network_provider.proxy.get_network_status(1)
    return {
        "current_epoch": raw_data.current_epoch,
        "current_round": raw_data.current_round,
        "current_block": raw_data.block_nonce,
        "current_block_timestamp": raw_data.block_timestamp
    }

In [ ]:
from utils.decoding_structures import METASTAKE_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, WrapperAddress, get_all_token_nonces_details_for_account

def user_farm_token_stats(user: Account):
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.farm_token, user.address.bech32(), context.network_provider.proxy)
    print(f'Account: {user.address.bech32()}')
    print(f'Looking for {metastaking_contract.farm_token} and {metastaking_contract.metastake_token} tokens')
    print(f'Farming Tokens in account:')
    for token in tokens_in_account:
        print(f'\t{token}')
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, user.address.bech32(), context.network_provider.proxy)
    print(f'Farm Tokens in account:')
    all_decoded_attributes = []
    for token in tokens_in_account:
        print(f'\t{token}')
        decoded_attributes = metastaking_contract.get_all_decoded_metastake_token_attributes_from_proxy(context.network_provider.proxy, user.address.bech32(), token['nonce'])
        print(f'\t\t{decoded_attributes}')
        all_decoded_attributes.append(decoded_attributes)
        
    return all_decoded_attributes

In [ ]:
from typing import Dict, Any, List, Tuple, Optional
from dataclasses import dataclass
from enum import Enum, auto
from collections import defaultdict

class DictType(Enum):
    USER_FARM_STATS_1 = auto()
    USER_STAKING_STATS_1 = auto()
    USER_2_FARM_STATS_1 = auto()
    USER_2_STAKING_STATS_1 = auto()
    PROXY_FARM_STATS_1 = auto()
    PROXY_STAKING_STATS_1 = auto()
    FARM_CONTRACT_STATS_1 = auto()
    STAKING_CONTRACT_STATS_1 = auto()
    TOKEN_ATTRS_1 = auto()
    USER_FARM_STATS_2 = auto()
    USER_STAKING_STATS_2 = auto()
    USER_2_FARM_STATS_2 = auto()
    USER_2_STAKING_STATS_2 = auto()
    PROXY_FARM_STATS_2 = auto()
    PROXY_STAKING_STATS_2 = auto()
    FARM_CONTRACT_STATS_2 = auto()
    STAKING_CONTRACT_STATS_2 = auto()
    TOKEN_ATTRS_2 = auto()
    OP1 = auto()
    OP2 = auto()
    OP3 = auto()
    OP4 = auto()
    OP5 = auto()

@dataclass
class DictComparison:
    before: Dict[str, Any]
    after: Dict[str, Any]
    description: str
    phase: str

class DictCollector:
    def __init__(self):
        self.reset()

    def reset(self):
        """Reset all collections for a new scenario"""
        self.before_upgrade: Dict[DictType, List[Tuple[Dict[str, Any], str]]] = defaultdict(list)
        self.after_upgrade: Dict[DictType, List[Tuple[Dict[str, Any], str]]] = defaultdict(list)
        self.current_phase = "before"  # "before" or "after"
    
    def set_phase(self, phase: str):
        """Set the current collection phase"""
        if phase not in ["before", "after"]:
            raise ValueError("Phase must be 'before' or 'after'")
        self.current_phase = phase

    def add(self, dict_type: DictType, dict_data: Dict[str, Any], description: str = ""):
        """Add a dictionary to the current phase collection"""
        if self.current_phase == "before":
            self.before_upgrade[dict_type].append((dict_data, description))
        else:
            self.after_upgrade[dict_type].append((dict_data, description))

    def _compare_dicts(self, dict1: Any, dict2: Any) -> Tuple[bool, Optional[str]]:
        """
        Compare two objects that can be either dictionaries or lists of dictionaries.
        Returns (is_equal, difference_description)
        """
        # Handle lists of dictionaries
        if isinstance(dict1, list) and isinstance(dict2, list):
            if len(dict1) != len(dict2):
                return False, f"Different list lengths: {len(dict1)} != {len(dict2)}"
            
            for i, (item1, item2) in enumerate(zip(dict1, dict2)):
                if isinstance(item1, (dict, list)) and isinstance(item2, (dict, list)):
                    is_equal, diff = self._compare_dicts(item1, item2)
                    if not is_equal:
                        return False, f"List item {i} difference: {diff}"
                elif item1 != item2:
                    return False, f"List item {i} mismatch: {item1} != {item2}"
            return True, None

        # Handle dictionaries
        if isinstance(dict1, dict) and isinstance(dict2, dict):
            if dict1.keys() != dict2.keys():
                missing_keys = set(dict1.keys()) - set(dict2.keys())
                extra_keys = set(dict2.keys()) - set(dict1.keys())
                return False, f"Different keys. Missing: {missing_keys}, Extra: {extra_keys}"

            for key in dict1:
                if isinstance(dict1[key], (dict, list)) and isinstance(dict2[key], (dict, list)):
                    is_equal, diff = self._compare_dicts(dict1[key], dict2[key])
                    if not is_equal:
                        return False, f"Nested difference at key '{key}': {diff}"
                elif dict1[key] != dict2[key]:
                    return False, f"Value mismatch for key '{key}': {dict1[key]} != {dict2[key]}"
            return True, None

        # Handle case where types don't match
        if type(dict1) != type(dict2):
            return False, f"Type mismatch: {type(dict1)} != {type(dict2)}"

        # Handle other cases
        return dict1 == dict2, f"Value mismatch: {dict1} != {dict2}" if dict1 != dict2 else None

    def compare_all(self) -> List[str]:
        """
        Compare all collected dictionary pairs and return a list of differences found.
        """
        differences = []

        for dict_type in DictType:
            before_list = self.before_upgrade[dict_type]
            after_list = self.after_upgrade[dict_type]

            # Check if we have matching pairs
            if len(before_list) != len(after_list):
                differences.append(
                    f"{dict_type.name}: Mismatched number of collections - "
                    f"Before: {len(before_list)}, After: {len(after_list)}"
                )
                continue

            # Compare each pair
            for i, ((before_dict, before_desc), (after_dict, after_desc)) in enumerate(zip(before_list, after_list)):
                is_equal, diff = self._compare_dicts(before_dict, after_dict)
                if not is_equal:
                    diff_msg = f"{dict_type.name} comparison failed"
                    if before_desc or after_desc:
                        diff_msg += f" (Before: {before_desc}, After: {after_desc})"
                    diff_msg += f": {diff}"
                    differences.append(diff_msg)

        return differences
    
    def print_collections(self):
        """
        Print a formatted view of all collections before and after upgrade.
        """
        print("\nCollections Summary:")
        print("=" * 80)

        for dict_type in DictType:            
            before_list = self.before_upgrade[dict_type]
            after_list = self.after_upgrade[dict_type]

            if not before_list and not after_list:
                continue

            print(f"\n{dict_type.name}:")
            print("-" * 40)

            print("\nBefore upgrade:")
            if not before_list:
                print("  No collections")
            for i, (data, desc) in enumerate(before_list, 1):
                print(f"  Collection {i}" + (f" ({desc})" if desc else ""))
                if isinstance(data, dict):
                    for key, value in data.items():
                        print(f"    {key}: {value}")
                else:
                    print(f"    {data}")

            print("\nAfter upgrade:")
            if not after_list:
                print("  No collections")
            for i, (data, desc) in enumerate(after_list, 1):
                print(f"  Collection {i}" + (f" ({desc})" if desc else ""))
                if isinstance(data, dict):
                    for key, value in data.items():
                        print(f"    {key}: {value}")
                else:
                    print(f"    {data}")

            print("\n" + "=" * 80)

# Example usage:
"""
collector = DictCollector()

# Before upgrade
collector.set_phase("before")
collector.add(DictType.USER_STATS, u1, "Initial user stats")
collector.add(DictType.CONTRACT_STATS, c1, "Initial contract stats")
collector.add(DictType.TOKEN_ATTRS, tk_attrs_1, "Initial token attributes")
collector.add(DictType.CLAIM_OPS, claim_ops_1, "Initial claim operations")

# After upgrade
collector.set_phase("after")
collector.add(DictType.USER_STATS, u3, "Post-upgrade user stats")
collector.add(DictType.CONTRACT_STATS, c3, "Post-upgrade contract stats")
collector.add(DictType.TOKEN_ATTRS, tk_attrs_2, "Post-upgrade token attributes")
collector.add(DictType.CLAIM_OPS, claim_ops_3, "Post-upgrade claim operations")

# Compare results
differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")

# Alternatively, at the end of your test
differences = collector.compare_all()
assert not differences, "\n".join(differences)

# Reset for next scenario
collector.reset()
"""

In [ ]:
def collect_general_test_data(index: int,collector: PhaseDictsCollector, user: Account, farm_contract: FarmContract, staking_contract: StakingContract):
    u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
    c1 = farm_contract.get_all_stats(context.network_provider.proxy)
    su1 = staking_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
    s1 = staking_contract.get_all_stats(context.network_provider.proxy)
    tk_attrs_1 = user_farm_token_stats(user)

    collector.add(f"USER_FARM_STATS_{index}", u1, f"Initial farm user stats")
    collector.add(f"FARM_CONTRACT_STATS_{index}", c1, f"Initial farm contract stats")
    collector.add(f"USER_STAKING_STATS_{index}", su1, f"Initial staking user stats")
    collector.add(f"STAKING_CONTRACT_STATS_{index}", s1, f"Initial staking contract stats")
    collector.add(f"TOKEN_ATTRS_{index}", tk_attrs_1, f"Initial token attributes")

def report_test_data(collector: PhaseDictsCollector, assert_no_differences: bool = True):
    collector.print_collections()
    
    differences = collector.compare_all()
    if differences:
        print("Found differences:")
        for diff in differences:
            print(f"- {diff}")
    else:
        print("All comparisons passed!")
        
    if assert_no_differences:
        assert not differences, "\n".join(differences)

In [ ]:
from copy import deepcopy
from pprint import pprint

def get_transfered_tokens_to_user(collector: PhaseDictsCollector, user: Account, dict_type: str):
    found_ops_before, found_ops_after = [], []
    
    def get_found_ops(container: list):
        found_ops = []
        for element in container:
            search_dict = {
                "action": "transfer",
                "sender": metastaking_contract.address,
                "receiver": user.address.bech32()
            }
            if all(item in element.items() for item in search_dict.items()):
                found_ops.append(element)
                # print(element)
        return found_ops
    
    # found_ops_before = get_found_ops(collector.before_upgrade[dict_type][0][0])
    # found_ops_after = get_found_ops(collector.after_upgrade[dict_type][0][0])
    found_ops_before = get_found_ops(collector.collections["before"][dict_type][0][0])
    found_ops_after = get_found_ops(collector.collections["after"][dict_type][0][0])
    return found_ops_before, found_ops_after

def compare_transfered_tokens_to_user(collector: PhaseDictsCollector, user: Account, dict_type: str):
    logger.info(f"Comparing for {dict_type}")

    found_ops_before, found_ops_after = get_transfered_tokens_to_user(collector, user, dict_type)

    if len(found_ops_before) < 3:
        logger.error(f"Not enough operations found for {dict_type} before upgrade!")
    if len(found_ops_after) < 3:
        logger.error(f"Not enough operations found for {dict_type} after upgrade!")
        
    for element in found_ops_before:
        # remove value from element, then compare with after_upgrade[dict_type][0][0]
        element_copy = deepcopy(element)
        amount = int(element_copy.pop("value"))
        for after_element in found_ops_after:
            if all(item in after_element.items() for item in element_copy.items()):
                compared_amount = int(after_element.get("value"))
                if amount < compared_amount:
                    logger.error(f"TOO MUCH {after_element.get('identifier')}: Before upgrade amount {amount} is less than {compared_amount} after upgrade!")
                # if difference is more than 1%
                elif abs(amount - compared_amount) / amount > 0.01:
                    logger.error(f"TOO LESS {after_element.get('identifier')}: Before upgrade amount {amount} is {abs(amount - compared_amount) / amount * 100}% different from {compared_amount} after upgrade!")
                else:
                    logger.info(f"OK {after_element.get('identifier')}: Before upgrade amount {amount} is {abs(amount - compared_amount) / amount * 100}% different from {compared_amount} after upgrade!")


In [ ]:
def init_pre_update_test(initial_blocks: int) -> Tuple[Any, List[Account], PhaseDictsCollector]:
    global found_accounts
    chain_sim, found_accounts = chain_sim_init()

    collector = PhaseDictsCollector()
    collector.set_phase("before")

    users = users_init()
    advance_blocks(3)
    consumed_blocks = 3

    collect_general_test_data(0, collector, users[0], farm_contract, staking_contract)
    collector.add(f"time_data_{0}", get_time_data(), "Time data")

    advance_blocks(initial_blocks - consumed_blocks)

    # collector = DictCollector()
    # collector.set_phase("before")

    return chain_sim, users, collector

def init_post_upgrade_test(initial_blocks: int, chain_sim: ChainSimulator, collector: PhaseDictsCollector):
    # input("Restart Chain Simulator then press Enter to continue...")
    
    chain_sim.stop()
    chain_sim_init()

    collector.set_phase("after")
    
    users = users_init()
    advance_blocks(1)
    consumed_blocks = 1

    farm_upgrade(False)  # eats 2 blocks
    # consumed_blocks += 2
    staking_upgrade(False)  # eats 2 blocks

    advance_blocks(2)
    consumed_blocks += 2

    collect_general_test_data(0, collector, users[0], farm_contract, staking_contract)
    collector.add(f"time_data_{0}", get_time_data(), "Time data")

    if initial_blocks < consumed_blocks:
        raise Exception(f"Initial blocks {initial_blocks} is less than consumed blocks {consumed_blocks}. Skewed results will occur.")
    
    block_diff = initial_blocks - consumed_blocks
    advance_blocks(block_diff)

    return chain_sim, users

def close_test(chain_sim: ChainSimulator, collector: PhaseDictsCollector, assert_no_differences: bool = True):
    report_test_data(collector, assert_no_differences)
    
    print("Chain simulator need to be stopped manually...")
    # chain_sim.stop()

In [ ]:
def dict_compare(d1, d2):
    print(d1)
    print(d2)
    d1_keys = set(d1.keys())
    d2_keys = set(d2.keys())
    shared_keys = d1_keys.intersection(d2_keys)
    added = d1_keys - d2_keys
    removed = d2_keys - d1_keys
    modified = {o : (d1[o], d2[o]) for o in shared_keys if d1[o] != d2[o]}
    same = set(o for o in shared_keys if d1[o] == d2[o])
    return added, removed, modified, same

def check_equal_dicts(dict1, dict2):
    """
    Compare two dictionaries, including nested dictionaries.
    
    Args:
    dict1 (dict): First dictionary to compare.
    dict2 (dict): Second dictionary to compare.
    
    Returns:
    bool: True if dictionaries are equal, False otherwise.
    """
    if dict1.keys() != dict2.keys():
        return False
    
    for key in dict1:
        if isinstance(dict1[key], dict) and isinstance(dict2[key], dict):
            if not check_equal_dicts(dict1[key], dict2[key]):
                return False
        elif dict1[key] != dict2[key]:
            return False
    
    return True

In [ ]:
def get_next_week_start_epoch(contract: FarmContract | StakingContract) -> int:
    first_week = contract.get_first_week_start_epoch(context.network_provider.proxy)
    current_week = contract.get_current_week(context.network_provider.proxy)
    next_week_at_epoch = first_week + current_week * 7

    print(f"Current epoch: {context.network_provider.proxy.get_network_status().current_epoch}")
    print(f"Current week: {current_week}")
    print(f"Next week at epoch: {next_week_at_epoch}")

    return next_week_at_epoch

def advance_to_next_week(contract: FarmContract | StakingContract):
    changing_epoch = get_next_week_start_epoch(contract)
    current_epoch = context.network_provider.proxy.get_network_status().current_epoch
    epochs_to_fast_forward = changing_epoch - current_epoch
    logger.info(f"Fast forwarding to {changing_epoch}")

    if config.CURRENT_ENV.value == "shadowfork4":
        from contracts.builtin_contracts import SFControlContract
        context.deployer_account.sync_nonce(context.network_provider.proxy)
        sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)
        sf_control_contract.epochs_fast_forward(context.deployer_account, context.network_provider.proxy, epochs_to_fast_forward, 9)

    elif config.CURRENT_ENV.value == "chainsim":
        chain_sim.advance_epochs_to_epoch(changing_epoch)
    else:
        raise Exception(f"Unknown environment: {config.CURRENT_ENV.value}")

    while context.network_provider.proxy.get_network_status().current_epoch < changing_epoch:
        sleep(6)
    logger.info(f"Fast forwarded to {changing_epoch}")

In [ ]:
import webbrowser

def open_tx_in_explorer(tx_hash: str, open_in_browser: bool = True):
    if not open_in_browser:
        return

    if config.CURRENT_ENV.value == "chainsim":
        webbrowser.open(f"https://custom-network.internal-explorer.multiversx.com/transactions/{tx_hash}")
    elif config.CURRENT_ENV.value == "shadowfork4":
        webbrowser.open(f"https://testnet-tc-shadowfork-four.internal-explorer.multiversx.com/transactions/{tx_hash}")


# SCENARIOS

Chain sim control

In [ ]:
chain_sim_stack = start_chain_sim_stack()

In [ ]:
stop_chain_sim_stack(chain_sim_stack)

Upgrade

In [ ]:
farm_upgrade()  # eats 2 blocks

staking_upgrade()  # eats 2 blocks

Init

In [ ]:
users = users_init()
user = users[0]
print(f"Using user: {user.address.bech32()}")

In [ ]:
user_farm_token_stats(user)

In [ ]:
from tools import chain_simulator_connector
user = users_init()[1]
user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
metastaking_token_attributes = metastaking_contract.get_decoded_metastake_token_attributes_from_proxy(context.network_provider.proxy, user.address.bech32(), user_tk_nonce)
farm_token = ESDTToken(metastaking_contract.farm_token, metastaking_token_attributes['lp_farm_token_nonce'], 0)
staking_token = ESDTToken(metastaking_contract.stake_token, metastaking_token_attributes['staking_farm_token_nonce'], 0)

# chain_simulator_connector.main(["--gateway", context.network_provider.proxy.url, "--token", farm_token.get_full_token_name()])
# chain_simulator_connector.main(["--gateway", context.network_provider.proxy.url, "--token", staking_token.get_full_token_name()])
print(farm_token.get_full_token_name())
print(staking_token.get_full_token_name())

Claim for user

In [ ]:
tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, user.address.bech32(), context.network_provider.proxy)
print(tokens_in_account)

In [ ]:
claim_for_user(user)
advance_blocks(1)

In [ ]:
user = users_init()[0]
claim_on_behalf_from_user(user)
advance_blocks(1)

In [ ]:
advance_blocks(1)

Claim on behalf

In [ ]:
user = users_init()[0]
permissions_hub_contract = deploy_permissions_hub()
metastaking_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
advance_blocks(1)

claim_on_behalf_from_user(user)
advance_blocks(1)

Claim compare

In [ ]:
show_in_browser = False

for i in range(5):
    user_index = i
    initial_blocks = 15
    in_week_blocks = 10
    next_week_blocks = 100*7

    def run_scenario():
        step = 1
        collect_general_test_data(step, collector, user, farm_contract, staking_contract)
        collector.add(f"time_data_{step}", get_time_data(), "Time data")
        step += 1

        logger.info(f"Claiming at block: {context.network_provider.proxy.get_network_status().block_nonce}")
        tx_hash = claim_for_user(user)
        advance_blocks(1)
        sleep(2)
        claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
        collector.add(f"claim_{step}", claim_ops_1, "First claim ops")
        open_tx_in_explorer(tx_hash, show_in_browser)

        collect_general_test_data(step, collector, user, farm_contract, staking_contract)
        collector.add(f"time_data_{step}", get_time_data(), "Time data")
        step += 1

        advance_blocks(in_week_blocks)
        logger.info(f"Claiming at block: {context.network_provider.proxy.get_network_status().block_nonce}")
        tx_hash = claim_for_user(user)
        advance_blocks(1)
        sleep(2)
        claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)
        collector.add(f"claim_{step}", claim_ops_2, "Second claim ops - same week")
        open_tx_in_explorer(tx_hash, show_in_browser)

        collect_general_test_data(step, collector, user, farm_contract, staking_contract)
        collector.add(f"time_data_{step}", get_time_data(), "Time data")
        step += 1

        advance_blocks(next_week_blocks)
        logger.info(f"Claiming at block: {context.network_provider.proxy.get_network_status().block_nonce}")
        tx_hash = claim_for_user(user)
        advance_blocks(1)
        sleep(2)
        claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)
        collector.add(f"claim_{step}", claim_ops_3, "Third claim ops - next week")
        open_tx_in_explorer(tx_hash, show_in_browser)

        collect_general_test_data(step, collector, user, farm_contract, staking_contract)
        collector.add(f"time_data_{step}", get_time_data(), "Time data")
        step += 1

        advance_blocks(in_week_blocks)
        logger.info(f"Claiming at block: {context.network_provider.proxy.get_network_status().block_nonce}")
        tx_hash = claim_for_user(user)
        advance_blocks(1)
        sleep(2)
        claim_ops_4 = context.network_provider.get_tx_operations(tx_hash, True)
        collector.add(f"claim_{step}", claim_ops_4, "Fourth claim ops - same week")
        open_tx_in_explorer(tx_hash, show_in_browser)

        collect_general_test_data(step, collector, user, farm_contract, staking_contract)
        collector.add(f"time_data_{step}", get_time_data(), "Time data")
        step += 1

        advance_blocks(next_week_blocks)
        logger.info(f"Claiming at block: {context.network_provider.proxy.get_network_status().block_nonce}")
        tx_hash = claim_for_user(user)
        advance_blocks(1)
        sleep(2)
        claim_ops_5 = context.network_provider.get_tx_operations(tx_hash, True)
        collector.add(f"claim_{step}", claim_ops_5, "Fifth claim ops - next week")
        open_tx_in_explorer(tx_hash, show_in_browser)
        
        collect_general_test_data(step, collector, user, farm_contract, staking_contract)
        collector.add(f"time_data_{step}", get_time_data(), "Time data")
        step += 1

        advance_blocks(next_week_blocks)
        logger.info(f"Claiming at block: {context.network_provider.proxy.get_network_status().block_nonce}")
        tx_hash = claim_for_user(user)
        advance_blocks(1)
        sleep(2)
        claim_ops_6 = context.network_provider.get_tx_operations(tx_hash, True)
        collector.add(f"claim_{step}", claim_ops_6, "Sixth claim ops - next week")
        open_tx_in_explorer(tx_hash, show_in_browser)
        
        collect_general_test_data(step, collector, user, farm_contract, staking_contract)
        collector.add(f"time_data_{step}", get_time_data(), "Time data")
        step += 1

        return step

    chain_sim, users, collector = init_pre_update_test(initial_blocks)
    user = users[user_index]

    run_scenario()

    ################################################################################
    chain_sim, users = init_post_upgrade_test(initial_blocks, chain_sim, collector)
    user = users[user_index]

    steps = run_scenario()

    # close_test(chain_sim, collector, False)

    for step in range(2, steps):
        compare_transfered_tokens_to_user(collector, user, f"claim_{step}")

    #raise Exception("Stop here")
    collector.reset()

In [ ]:
print(staking_contract.get_next_week_start_epoch(context.network_provider.proxy))
print(context.network_provider.proxy.get_network_status().current_epoch)

In [ ]:
get_transfered_tokens_to_user(collector, user, "claim_1")

In [ ]:
print(collector.collections["after"]["claim_2"])

In [ ]:
from utils.contract_data_fetchers import BaseFarmContractDataFetcher

print(staking_contract.address)
fetcher = BaseFarmContractDataFetcher(WrapperAddress(staking_contract.address), context.network_provider.proxy.url)
print(fetcher.get_data("getPerSecondRewardAmount"))

print(3000000000000000000 // 6)

upgrade max apr integrity calculation

In [ ]:
# amount * &max_apr / MAX_PERCENT / BLOCKS_IN_YEAR
# division_safety * amount * &max_apr / MAX_PERCENT / SECONDS_IN_YEAR

ts_diff = 60
farm_supply = staking_contract.get_farm_token_supply(context.network_provider.proxy)
division_safety = staking_contract.get_division_safety_constant(context.network_provider.proxy)
max_apr = staking_contract.get_max_apr(context.network_provider.proxy)

a = farm_supply * max_apr // 10_000 // 31_536_000 // 6
b = division_safety * farm_supply * max_apr // 10_000 // 31_536_000

a_aggregated = a * (ts_diff // 6)
b_aggregated = b * ts_diff // division_safety

print(f'old: {a_aggregated}')
print(f'new: {b_aggregated}')

In [ ]:
global found_accounts
chain_sim, found_accounts = chain_sim_init()

collector = PhaseDictsCollector()
collector.set_phase("before")

users = users_init()
advance_blocks(3)
consumed_blocks = 3

claim_for_user(users[0])
advance_blocks(1)
sleep(2)
collect_general_test_data(0, collector, users[0], farm_contract, staking_contract)
collector.add(f"time_data_{0}", get_time_data(), "Time data")

advance_blocks(10)

claim_for_user(users[0])
advance_blocks(1)
sleep(2)
collect_general_test_data(1, collector, users[0], farm_contract, staking_contract)
collector.add(f"time_data_{1}", get_time_data(), "Time data")

# collector = DictCollector()
# collector.set_phase("before")


chain_sim.stop()
chain_sim_init()

collector.set_phase("after")

users = users_init()
advance_blocks(3)
consumed_blocks = 3

claim_for_user(users[0])
advance_blocks(1)
sleep(2)
collect_general_test_data(0, collector, users[0], farm_contract, staking_contract)
collector.add(f"time_data_{0}", get_time_data(), "Time data")

advance_blocks(10)

farm_upgrade(False)  # eats 2 blocks
# consumed_blocks += 2
staking_upgrade(False)  # eats 2 blocks

advance_blocks(1)
consumed_blocks += 2

collect_general_test_data(1, collector, users[0], farm_contract, staking_contract)
collector.add(f"time_data_{1}", get_time_data(), "Time data")

In [ ]:
# report_test_data(collector, True)
for i in range(2):
    print(collector.collections["before"][f"FARM_CONTRACT_STATS_{i}"])
    print(collector.collections["after"][f"FARM_CONTRACT_STATS_{i}"])
    print("-"*100)

In [ ]:
data_0, _ = collector.collections["before"]["STAKING_CONTRACT_STATS_0"][0]
data_1, _ = collector.collections["before"]["STAKING_CONTRACT_STATS_1"][0]
diff_before = data_1.get("accumulated_rewards") - data_0.get("accumulated_rewards")
print(f'diff before: {diff_before}')

data_0, _ = collector.collections["after"]["STAKING_CONTRACT_STATS_0"][0]
data_1, _ = collector.collections["after"]["STAKING_CONTRACT_STATS_1"][0]
diff_after = data_1.get("accumulated_rewards") - data_0.get("accumulated_rewards")
print(f'diff after: {diff_after}')
print(f'rpb: {diff_after // 11}')

In [ ]:
from pprint import pprint
found_ops_before, found_ops_after = get_transfered_tokens_to_user(collector, user, "claim_2")
pprint(found_ops_before)
pprint(found_ops_after)
compare_transfered_tokens_to_user(collector, user, f"claim_2")

In [ ]:
for i in range(7):
    print(collector.collections["before"][f"STAKING_CONTRACT_STATS_{i}"])
    print(collector.collections["after"][f"STAKING_CONTRACT_STATS_{i}"])
    print("-"*100)

In [ ]:
for i in range(7):
    print(collector.collections["before"][f"FARM_CONTRACT_STATS_{i}"])
    print(collector.collections["after"][f"FARM_CONTRACT_STATS_{i}"])
    print("-"*100)
#50.39999999999999999999

In [ ]:
for i in range(7):
    print(collector.collections["before"][f"time_data_{i}"])
    print(collector.collections["after"][f"time_data_{i}"])
    print("-"*100)

Claim compare on multiple users

In [ ]:
show_in_browser = False

initial_blocks = 15
in_week_blocks = 10
next_week_blocks = 100*7

users_to_test = 10

def run_scenario():
    step = 1
    collect_general_test_data(step, collector, users[0], farm_contract, staking_contract)
    collector.add(f"time_data_{step}", get_time_data(), "Time data")
    step += 1

    logger.info(f"Claiming at block: {context.network_provider.proxy.get_network_status().block_nonce}")
    for i in range(users_to_test):
        user = users[i]
        tx_hash = claim_for_user(user)
        advance_blocks(1)
        sleep(2)
        claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
        collector.add(f"claim_{step}_{i}", claim_ops_1, "First claim ops")
        open_tx_in_explorer(tx_hash, show_in_browser)

    collect_general_test_data(step, collector, users[0], farm_contract, staking_contract)
    collector.add(f"time_data_{step}", get_time_data(), "Time data")
    step += 1

    advance_blocks(in_week_blocks)
    logger.info(f"Claiming at block: {context.network_provider.proxy.get_network_status().block_nonce}")
    for i in range(users_to_test):
        user = users[i]
        tx_hash = claim_for_user(user)
        advance_blocks(1)
        sleep(2)
        claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)
        collector.add(f"claim_{step}_{i}", claim_ops_2, "Second claim ops - same week")
        open_tx_in_explorer(tx_hash, show_in_browser)

    collect_general_test_data(step, collector, users[0], farm_contract, staking_contract)
    collector.add(f"time_data_{step}", get_time_data(), "Time data")
    step += 1

    farm_next_week_start_epoch = farm_contract.get_next_week_start_epoch(context.network_provider.proxy)
    staking_next_week_start_epoch = staking_contract.get_next_week_start_epoch(context.network_provider.proxy)
    chain_sim.advance_epochs_to_epoch(max(farm_next_week_start_epoch, staking_next_week_start_epoch))
    logger.info(f"Claiming at block: {context.network_provider.proxy.get_network_status().block_nonce}")
    for i in range(users_to_test):
        user = users[i]
        tx_hash = claim_for_user(user)
        advance_blocks(1)
        sleep(2)
        claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)
        collector.add(f"claim_{step}_{i}", claim_ops_3, "Third claim ops - next week")
        open_tx_in_explorer(tx_hash, show_in_browser)

    collect_general_test_data(step, collector, users[0], farm_contract, staking_contract)
    collector.add(f"time_data_{step}", get_time_data(), "Time data")
    step += 1

    farm_next_week_start_epoch = farm_contract.get_next_week_start_epoch(context.network_provider.proxy)
    staking_next_week_start_epoch = staking_contract.get_next_week_start_epoch(context.network_provider.proxy)
    chain_sim.advance_epochs_to_epoch(max(farm_next_week_start_epoch, staking_next_week_start_epoch))
    logger.info(f"Claiming at block: {context.network_provider.proxy.get_network_status().block_nonce}")
    for i in range(users_to_test):
        user = users[i]
        tx_hash = claim_for_user(user)
        advance_blocks(1)
        sleep(2)
        claim_ops_4 = context.network_provider.get_tx_operations(tx_hash, True)
        collector.add(f"claim_{step}_{i}", claim_ops_4, "Fourth claim ops - next week")
        open_tx_in_explorer(tx_hash, show_in_browser)

    collect_general_test_data(step, collector, users[0], farm_contract, staking_contract)
    collector.add(f"time_data_{step}", get_time_data(), "Time data")
    step += 1

    return step

chain_sim, users, collector = init_pre_update_test(initial_blocks)

run_scenario()

################################################################################
chain_sim, users = init_post_upgrade_test(initial_blocks, chain_sim, collector)

steps = run_scenario()

# close_test(chain_sim, collector, False)

for step in range(2, steps):
    for i in range(users_to_test):
        compare_transfered_tokens_to_user(collector, users[i], f"claim_{step}_{i}")

#raise Exception("Stop here")
# collector.reset()

Exit compare

In [ ]:
show_in_browser = False

initial_blocks = 15
in_week_blocks = 10
next_week_blocks = 100*7

users_to_test = 10

def run_scenario():
    step = 1
    collect_general_test_data(step, collector, users[0], farm_contract, staking_contract)
    collector.add(f"time_data_{step}", get_time_data(), "Time data")
    step += 1

    logger.info(f"Exiting at block: {context.network_provider.proxy.get_network_status().block_nonce}")
    for i in range(users_to_test//2):
        user = users[i]
        tx_hash = exit_for_user(user)
        advance_blocks(1)
        sleep(2)
        exit_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
        collector.add(f"exit_{step}_{i}", exit_ops_1, "First exit ops")
        open_tx_in_explorer(tx_hash, show_in_browser)

    collect_general_test_data(step, collector, users[0], farm_contract, staking_contract)
    collector.add(f"time_data_{step}", get_time_data(), "Time data")
    step += 1

    farm_next_week_start_epoch = farm_contract.get_next_week_start_epoch(context.network_provider.proxy)
    staking_next_week_start_epoch = staking_contract.get_next_week_start_epoch(context.network_provider.proxy)
    chain_sim.advance_epochs_to_epoch(max(farm_next_week_start_epoch, staking_next_week_start_epoch))
    logger.info(f"Exiting at block: {context.network_provider.proxy.get_network_status().block_nonce}")
    for i in range(users_to_test//2, users_to_test):
        user = users[i]
        tx_hash = exit_for_user(user)
        advance_blocks(1)
        sleep(2)
        exit_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)
        collector.add(f"exit_{step}_{i}", exit_ops_2, "Second exit ops - next week")
        open_tx_in_explorer(tx_hash, show_in_browser)

    collect_general_test_data(step, collector, users[0], farm_contract, staking_contract)
    collector.add(f"time_data_{step}", get_time_data(), "Time data")
    step += 1

    return step

chain_sim, users, collector = init_pre_update_test(initial_blocks)

run_scenario()

################################################################################
chain_sim, users = init_post_upgrade_test(initial_blocks, chain_sim, collector)

steps = run_scenario()

# close_test(chain_sim, collector, False)

for step in range(2, steps):
    for i in range(users_to_test):
        compare_transfered_tokens_to_user(collector, users[i], f"exit_{step}_{i}")

#raise Exception("Stop here")
# collector.reset()

In [ ]:
for j in range(users_to_test):
    print(collector.collections["before"][f"exit_{3}_{j}"])
    print(collector.collections["after"][f"exit_{3}_{j}"])
    print("-"*100)

Enter farm compare

In [ ]:
user_index = 0
initial_blocks = 15

chain_sim, users, collector = init_pre_update_test(initial_blocks)
user = users[user_index]

collect_general_test_data(0, collector, user, farm_contract, staking_contract)

tx_hash = enter_farm_for_user(user)
advance_blocks(5)

sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
collect_general_test_data(1, collector, user, farm_contract, staking_contract)
collector.add(DictType.OP1, claim_ops_1, "Initial enter ops")

################################################################################
chain_sim, users, permissions_hub_contract, dummy_proxy_contract = init_post_upgrade_test(initial_blocks, chain_sim, collector)
user = users[user_index]

collect_general_test_data(0, collector, user, farm_contract, staking_contract)

tx_hash = enter_farm_for_user(user)
advance_blocks(5)

sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)
collect_general_test_data(1, collector, user, farm_contract, staking_contract)
collector.add(DictType.OP1, claim_ops_3, "Ending enter ops")

close_test(chain_sim, collector)